In [ ]:
import pandas as pd
import numpy as np
import pydot as pydot
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
import plotly.express as px
plt.style.use("ggplot")
rcParams['figure.figsize'] = (12,6)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
from sklearn import metrics
from sklearn import tree
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import brier_score_loss
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow import keras
from sklearn.metrics import classification_report
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from imblearn import over_sampling
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn import preprocessing
import tensorflow as tf

In [ ]:
def idx_cm(y_pred, y_test, pos, neg):
    fp_rows = []
    fn_rows = []
    tp_rows = []
    tn_rows = []

    for i in range(len(y_pred)):
        if y_pred[i] == pos and y_test[i] == neg:
            fp_rows.append(i)
        elif y_pred[i] == pos and y_test[i] == pos:
            tp_rows.append(i)
        elif y_pred[i] == neg and y_test[i] == pos:
            fn_rows.append(i)
        elif y_pred[i] == neg and y_test[i] == neg:
            tn_rows.append(i)
    fp_rows = pd.DataFrame(fp_rows)
    tp_rows = pd.DataFrame(tp_rows)
    fn_rows = pd.DataFrame(fn_rows)
    tn_rows = pd.DataFrame(tn_rows)
    cm = [fp_rows, tp_rows, fn_rows, tn_rows]

def hypertune_threshold(model, start, stop, X_test, y_test, pos, neg):
    threshold = np.linspace(start = start, stop = stop, num = 100)
    f1 = np.zeros(threshold.size)
    predictions = model.predict_proba(X_test)
    for i in np.arange(threshold.size):
        probs = np.where(predictions[:,pos]>=threshold[i], pos, neg)
        f1[i] = f1_score(np.ravel(y_test), probs, pos_label=pos)
    final_probs = np.where(predictions[:,pos]>=threshold[np.argmax(f1)], pos, neg)
    return print(threshold[np.argmax(f1)], f1_score(np.ravel(y_test), final_probs, pos_label=pos)), final_probs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

root_path = '/content/drive/My Drive/'

Mounted at /content/drive


In [ ]:
X_train = pd.read_csv(root_path+'X_train_lvl1.csv')
y_train = pd.read_csv(root_path+'y_train_lvl1.csv')
X_test = pd.read_csv(root_path+'X_test_lvl1.csv')
y_test = pd.read_csv(root_path+'y_test_lvl1.csv')
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(122067, 66) (122067, 1) (40690, 66) (40690, 1)


In [ ]:
X_train.info()

In [ ]:
y_train = np.array(y_train)
y_test = np.array(y_test)

y_train = ~y_train+2
y_test = ~y_test+2

X_train = np.array(X_train)
X_test = np.array(X_test)

ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

print('Training labels shape:', y_resampled.shape)

print('Training features shape:', X_resampled.shape)

# Sklearn XGBoost

In [ ]:
model = xgb.XGBClassifier(n_estimators=30000, objective = 'binary:logistic',
                                   learning_rate = 0.0001,
                                   gamma = 0.5, reg_lambda = 0.5,
                                   subsample = 0.8, n_jobs = -1, random_state = 50, max_leaves=10, early_stopping_rounds = 1000, tree_method = "gpu_hist")
evalset = [(X_resampled, y_resampled), (X_test,y_test)]

In [ ]:
max_train_size = len(X_train)

train_sizes = []
auc_roc_scores_test = []
f1_scores_test = []
brier_scores_test = []
loss_scores_test = []
auc_roc_scores_train = []
f1_scores_train = []
brier_scores_train = []
loss_scores_train = []

for train_size in range(500, max_train_size + 1, 1000):
    # Initialize the cross-validation object
    kfold = StratifiedKFold(n_splits=5)

    fold_auc_roc_scores_test = []
    fold_f1_scores_test = []
    fold_brier_scores_test = []
    fold_loss_scores_test = []
    fold_auc_roc_scores_train = []
    fold_f1_scores_train = []
    fold_brier_scores_train = []
    fold_loss_scores_train = []

    # Split the data into train and test sets using cross-validation
    for train_index, test_index in kfold.split(X_train[:train_size], y_train[:train_size]):
        X_train_fold, X_test_fold = X_train[:train_size][train_index], X_train[:train_size][test_index]
        y_train_fold, y_test_fold = y_train[:train_size][train_index], y_train[:train_size][test_index]

    # Normalize data
        scaler = preprocessing.MinMaxScaler((0,1))
        X_train_fold = scaler.fit_transform(X_train_fold)
        X_test_fold = scaler.transform(X_test_fold)
    # Resample train set
        ros = RandomUnderSampler(random_state=42)
        train_resampled, y_resampled = ros.fit_resample(X_train_fold, y_train_fold)

    # Fit the model
        model.fit(train_resampled, y_resampled, eval_metric='auc', eval_set=[(train_resampled, y_resampled), (X_test_fold,y_test_fold)], verbose=200)

   # Make predictions
        y_pred_test = model.predict_proba(X_test_fold)[:,1]
        y_pred_train = model.predict_proba(X_train_fold)[:,1]

        # Calculate and store the AUC-ROC score
        auc_roc_test = roc_auc_score(y_test_fold, y_pred_test)
        auc_roc_train = roc_auc_score(y_train_fold, y_pred_train)
        fold_auc_roc_scores_test.append(auc_roc_test)
        fold_auc_roc_scores_train.append(auc_roc_train)

        # Calculate and store the F1 score
        y_pred_binary_test = np.round(y_pred_test)
        y_pred_binary_train = np.round(y_pred_train)
        f1_test = f1_score(y_test_fold, y_pred_binary_test)
        f1_train = f1_score(y_train_fold, y_pred_binary_train)
        fold_f1_scores_test.append(f1_test)
        fold_f1_scores_train.append(f1_train)

        # Calculate and store the Brier loss
        brier_loss_test = brier_score_loss(y_test_fold, y_pred_test)
        fold_brier_scores_test.append(brier_loss_test)
        brier_loss_train = brier_score_loss(y_train_fold, y_pred_train)
        fold_brier_scores_train.append(brier_loss_train)

        # Calculate and store the loss
        loss_test = log_loss(y_test_fold, y_pred_test)
        fold_loss_scores_test.append(loss_test)
        loss_train = log_loss(y_train_fold, y_pred_train)
        fold_loss_scores_train.append(loss_train)

    # Calculate the average scores across all folds for the current train set size
    average_auc_roc_test = np.mean(fold_auc_roc_scores_test)
    average_f1_test = np.mean(fold_f1_scores_test)
    average_brier_test = np.mean(fold_brier_scores_test)
    average_loss_test = np.mean(fold_loss_scores_test)
    average_auc_roc_train = np.mean(fold_auc_roc_scores_train)
    average_f1_train = np.mean(fold_f1_scores_train)
    average_brier_train = np.mean(fold_brier_scores_train)
    average_loss_train = np.mean(fold_loss_scores_train)

# Append the train set size and average scores to the respective lists
    train_sizes.append(train_size)
    auc_roc_scores_test.append(average_auc_roc_test)
    f1_scores_test.append(average_f1_test)
    brier_scores_test.append(average_brier_test)
    loss_scores_test.append(average_loss_test)
    auc_roc_scores_train.append(average_auc_roc_train)
    f1_scores_train.append(average_f1_train)
    brier_scores_train.append(average_brier_train)
    loss_scores_train.append(average_loss_train)


In [ ]:
model.fit(X_resampled, y_resampled, eval_metric='logloss', eval_set=evalset, verbose=200)

In [ ]:
y_pred = model.predict(X_test)
cm = idx_cm(y_pred, y_test, 1,0)

In [ ]:
sns.kdeplot(model.predict_proba(X_test)[:,1])

In [ ]:
sns.kdeplot(y_prob[tn_rows])
plt.xlabel("Probability of being 1")
plt.legend('',frameon=False)

In [ ]:
X_test[["OWNSHIP", "AREA", "CAR_AVG", "SOCGRP_AVG", "MARI_STATUS", "WEALTH_AVG", "HOU_INCO_AVG", "USAGE_Apartment", "EDU_AVG"]].iloc[fp_rows[0]].describe()

In [ ]:
print(model.feature_importances_)
# plot
plt.bar(range(len(model.feature_importances_)), model.feature_importances_)
plt.show()

In [ ]:
# plot learning curves
plt.plot(results['validation_0']['logloss'], label='train')
plt.plot(results['validation_1']['logloss'], label='test')
# show the legend
plt.legend()
plt.title("XGBoost")
plt.xlabel("Epochs")
plt.ylabel("Loss")
# show the plot
plt.show()

In [ ]:
f1, final_probs = hypertune_threshold(model, 0.1, 0.6, test, y_test, 0, 1)

In [ ]:
fig, ax = plt.subplots()
ax= sns.lineplot(x=train_sizes, y=auc_roc_scores_test, label = "test")
ax1 = sns.lineplot(x=train_sizes, y=auc_roc_scores_train, label = "train")
ax.set(xlabel='Number of observations', ylabel='ROC_AUC')

In [ ]:
fig, ax = plt.subplots()
ax= sns.lineplot(x=train_sizes, y=f1_scores_test, label = "test")
ax1 = sns.lineplot(x=train_sizes, y=f1_scores_train, label = "train")
ax.set(xlabel='Number of observations', ylabel='F1')

In [ ]:
fig, ax = plt.subplots()
ax= sns.lineplot(x=train_sizes, y=loss_scores_test, label = "test")
ax1 = sns.lineplot(x=train_sizes, y=loss_scores_train, label = "train")
ax.set(xlabel='Number of observations', ylabel='Logloss')

In [ ]:
fig, ax = plt.subplots()
ax= sns.lineplot(x=train_sizes, y=brier_scores_test, label = "test")
ax1 = sns.lineplot(x=train_sizes, y=brier_scores_train, label = "train")
ax.set(xlabel='Number of observations', ylabel='Brier score')

In [ ]:
cm = metrics.confusion_matrix(y_test,model.predict(X_test))
fig = plt.figure(figsize=(9, 9))
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square=True, cmap='Blues_r');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
plt.title(XGBoost, size=15);